In [1]:
import ifcopenshell
import ifcopenshell.api
import pandas as pd

ifc_path_in = '/Users/fnoi/Library/CloudStorage/OneDrive-TUM/2025_egice_noichl_forth/all_beams.ifc'
ifc_path_out = '/Users/fnoi/Library/CloudStorage/OneDrive-TUM/2025_egice_noichl_forth/auto_test_v0.ifc'

epea_path = '/Users/fnoi/Library/CloudStorage/OneDrive-TUM/2025_egice_noichl_forth/epea_src.csv'
epea = pd.read_csv(epea_path, header=0)
epea['crit'] = epea['crit'].str.replace('*', '').str.replace(' ', '')

In [3]:
epea

,madaster_uid,external_id,name_ger,crit
0,0a88355b-ff78-4187-b8a3-7ef091063142,L3735,Stahlträger INP Profil 500,INP500
1,244ed9fa-31f3-4ec7-af25-223f590e364a,L3635,Stahlträger HEA Profil 360,HEA360
2,334de5b6-6cb8-4b6d-b7c1-159bffd41755,L3718,Stahlträger INP Profil 120,INP120
3,381aae56-c371-4e95-aa07-a4be08d45830,L3744,Stahlträger UPE Profil 200,UPE200
4,39c22785-e08a-4da3-b6c7-413da77aecb2,L3650,Stahlträger HEB Profil 180,HEB180
...,...,...,...,...
138,ae836c4c-29d1-4762-8ece-2d662d1ed695,L3646,Stahlträger HEB Profil 100,HEB100
139,cc85364a-1f3d-43df-95ca-75ad2b6b4b7e,L3701,Stahlträger IPE Profil 180,IPE180
140,d55c7f38-4a8e-4a86-b5ab-91ca8df5df9a,L3767,Stahlträger UNP Profil 320,UNP320
141,d639897e-2348-4df6-a306-3b860482ea61,L3758,Stahlträger UNP Profil 140,UNP140


In [ ]:
ifc_file = ifcopenshell.open(ifc_path_in)

# Get all beam type objects
beam_types = ifc_file.by_type("IfcBeamType")
print(f"Found {len(beam_types)} beam type objects")

# Get owner history if it exists
owner_history = None
if ifc_file.by_type("IfcOwnerHistory"):
    owner_history = ifc_file.by_type("IfcOwnerHistory")[0]

# Iterate over each beam type
for beam_type in beam_types:
    beam_name = beam_type.Name if beam_type.Name else f"Unnamed-{beam_type.id()}"
    print(f"Processing beam type: {beam_name}")

    profiles = {}
    epea_name = beam_name.split('_')[1]
    print(f'looking for {epea_name}')

    deb = epea[epea.crit == epea_name]
    if deb.empty:
        print(f"  No match found for {beam_name}, setting MaterialOrProductId to 'Unknown'")
        deb = 'Unknown'
    else:
        deb = deb['madaster_uid'].values[0]
    print(f"  Found MaterialOrProductId: {deb}")
    
    # Profile name for MaterialOrProductName
    profile_name = f"Steel Beam {epea_name}"

    # Create a new property set with owner history
    pset = ifc_file.create_entity(
        "IfcPropertySet",
        GlobalId=ifcopenshell.guid.new(),
        OwnerHistory=owner_history,  # Link to owner history
        Name="CPset_Madaster",
        Description=None,
        HasProperties=[]
    )

    # Create relationship to beam type with owner history
    rel = ifc_file.create_entity(
        "IfcRelDefinesByProperties",
        GlobalId=ifcopenshell.guid.new(),
        OwnerHistory=owner_history,  # Link to owner history
        RelatedObjects=[beam_type],
        RelatingPropertyDefinition=pset
    )
    print(f"  Created new CPset_Madaster for {deb} with owner history")

    # Create properties list
    props = []

    # Create Classification property
    classification_prop = ifc_file.create_entity(
        "IfcPropertySingleValue",
        Name="Classification",
        Description="",
        NominalValue=ifc_file.create_entity("IfcText", "351")
    )
    props.append(classification_prop)
    print(f"  Added Classification: 351")

    # Create Phase property
    phase_prop = ifc_file.create_entity(
        "IfcPropertySingleValue",
        Name="Phase",
        Description="",
        NominalValue=ifc_file.create_entity("IfcText", "Bestand")
    )
    props.append(phase_prop)
    print(f"  Added Phase: Bestand")

    # Create DetachabilityIntersection property - Fixed name
    intersection_prop = ifc_file.create_entity(
        "IfcPropertySingleValue",
        Name="DetachabilityIntersection",
        Description="",
        NominalValue=ifc_file.create_entity("IfcText", "None")
    )
    props.append(intersection_prop)
    print(f"  Added DetachabilityIntersection: None")

    # Create MaterialOrProductName property
    product_name_prop = ifc_file.create_entity(
        "IfcPropertySingleValue",
        Name="MaterialOrProductName",
        Description="",
        NominalValue=ifc_file.create_entity("IfcText", profile_name)
    )
    props.append(product_name_prop)
    print(f"  Added MaterialOrProductName: {profile_name}")

    # Create DetachabilityConnectionTypeDetail property
    connection_type_prop = ifc_file.create_entity(
        "IfcPropertySingleValue",
        Name="DetachabilityConnectionTypeDetail",
        Description="",
        NominalValue=ifc_file.create_entity("IfcText", "HardChemicalConnection")
    )
    props.append(connection_type_prop)
    print(f"  Added DetachabilityConnectionTypeDetail: HardChemicalConnection")

    # Create MaterialOrProductId property
    material_prop = ifc_file.create_entity(
        "IfcPropertySingleValue",
        Name="MaterialOrProductId",
        Description="",
        NominalValue=ifc_file.create_entity("IfcText", deb)
    )
    props.append(material_prop)
    print(f"  Added MaterialOrProductId: {deb}")

    # Create DetachabilityProductEdge property
    product_edge_prop = ifc_file.create_entity(
        "IfcPropertySingleValue",
        Name="DetachabilityProductEdge",
        Description="",
        NominalValue=ifc_file.create_entity("IfcText", "Overlapping")
    )
    props.append(product_edge_prop)
    print(f"  Added DetachabilityProductEdge: Overlapping")

    # Create DetachabilityAccessibility property
    accessibility_prop = ifc_file.create_entity(
        "IfcPropertySingleValue",
        Name="DetachabilityAccessibility",
        Description="",
        NominalValue=ifc_file.create_entity("IfcText", "Accessible")
    )
    props.append(accessibility_prop)
    print(f"  Added DetachabilityAccessibility: Accessible")

    # Create MaterialOrProductRatio property - Fixed
    ratio_prop = ifc_file.create_entity(
        "IfcPropertySingleValue",
        Name="MaterialOrProductRatio",
        Description="",
        NominalValue=ifc_file.create_entity("IfcText", "1")
    )
    props.append(ratio_prop)
    print(f"  Added MaterialOrProductRatio: 1")

    # Set all properties at once
    pset.HasProperties = props

# Save the modified file
ifc_file.write(ifc_path_out)
print(f"Saved modified file as: {ifc_path_out}")

Found 14 beam type objects
Processing beam type: T_HEA140
looking for HEA140
  Found MaterialOrProductId: 6a04ca4b-3bee-4717-b82c-4cadcfb5f187
  Created new CPset_Madaster for 6a04ca4b-3bee-4717-b82c-4cadcfb5f187 with owner history
  Added Classification: 351
  Added Phase: Bestand
  Added DetachabilityIntersection: None
  Added MaterialOrProductName: Steel Beam HEA140
  Added DetachabilityConnectionTypeDetail: HardChemicalConnection
  Added MaterialOrProductId: 6a04ca4b-3bee-4717-b82c-4cadcfb5f187
  Added DetachabilityProductEdge: Overlapping
  Added DetachabilityAccessibility: Accessible
  Added MaterialOrProductRatio: 1
Saved modified file as: /Users/fnoi/Library/CloudStorage/OneDrive-TUM/2025_egice_noichl_forth/auto_test_v0.ifc
